In [91]:
import pandas as pd
import numpy as np

In [92]:
# Load datasets
db1 = pd.read_excel('coem_authors_enriched_cleaned.xlsx')
db = pd.read_excel('coem_authors_enriched.xlsx')
# db2 = pd.read_csv('embeddings2.csv')
# db3 = pd.read_csv('embeddings3.csv')

In [93]:
db1.columns

Index(['Título', 'Tiempo de Lectura (min)', 'Primer Nombre Autor',
       'Apellido Autor', 'Nombre Completo Autor', 'País', 'Género',
       'Año de nacimiento', 'Año de muerte', 'Autores relacionados',
       'Categorías', 'author_uuid', 'Imagen', 'Audio'],
      dtype='object')

In [94]:
authors = db1[['Nombre Completo Autor', 'Autores relacionados']]
authors.head()

,Nombre Completo Autor,Autores relacionados
0,Pío Baroja,"Antonio Machado,Benito Pérez Galdós,Camilo Jos..."
1,Isaac Asimov,"Arthur C. Clarke,Kurt Vonnegut"
2,Mario Benedetti,"Gabriela Mistral,José Saramago,Pablo Neruda,Ru..."
3,Virgilio Pinera,"Charles Baudelaire,Cintio Vitier,Eliseo Diego,..."
4,Esopo,NaN


In [95]:
authors = db1[['Nombre Completo Autor', 'Autores relacionados']]
authors.head()

,Nombre Completo Autor,Autores relacionados
0,Pío Baroja,"Antonio Machado,Benito Pérez Galdós,Camilo Jos..."
1,Isaac Asimov,"Arthur C. Clarke,Kurt Vonnegut"
2,Mario Benedetti,"Gabriela Mistral,José Saramago,Pablo Neruda,Ru..."
3,Virgilio Pinera,"Charles Baudelaire,Cintio Vitier,Eliseo Diego,..."
4,Esopo,NaN


In [96]:
# prompt: Convert all nan on db1['linked_authors'] and db1['Author_name'] to empty strings to all the column is strings

db1['Autores relacionados'] = db1['Autores relacionados'].fillna('')
db1['Nombre Completo Autor'] = db1['Nombre Completo Autor'].fillna('')
db1['Género'] = db1['Género'].fillna('Desconocido')
db1['Año de nacimiento'] = db1['Año de nacimiento'].fillna('Desconocido')
db1['Año de muerte'] = db1['Año de muerte'].fillna('Desconocido')
db1['Título'] = db1['Título'].fillna('Desconocido')
db1['País'] = db1['País'].fillna('Desconocido')


db1['Año de nacimiento'] = db1['Año de nacimiento'].astype(str).str.replace(".0","")
db1['Año de muerte'] = db1['Año de muerte'].astype(str).str.replace(".0","")
db1['Autores relacionados'] = db1['Autores relacionados'].str.replace("nan","")
db1['Nombre Completo Autor'] = db1['Nombre Completo Autor'].str.replace("nan","")
db1['Año de nacimiento'] = db1['Año de nacimiento'].str.replace("nan","Desconocido")
db1['Año de muerte'] = db1['Año de muerte'].str.replace("nan","Desconocido")

db1['Título'] = db1['Título'].str.replace("nan","Desconocido")
db1['País'] = db1['País'].str.replace("nan","Desconocido")

In [98]:
# prompt: A list of unique countries in db1

unique_countries = db1['País'].unique().tolist()

last_countries=["Desconocido"]
last_countries.extend(unique_countries)
unique_countries=last_countries.copy()
# Get index of country and default 0
get_country_index = lambda country: unique_countries.index(country) if country in unique_countries else 0
country_index = get_country_index('España')
print(unique_countries)
print(country_index)


['Desconocido', 'España', 'Estados Unidos', 'Uruguay', 'Cuba', 'Grecia', 'Francia', 'Argentina', 'Desconocido', 'Italia', 'India', 'Mexico', 'Rusia', 'Inglaterra', 'África', 'Brasil', 'Portugal', 'Arabia', 'Guatemala', 'Dinamarca', 'Irlanda', 'Colombia', 'Peru', 'Alemania', 'Checoslovaquia', 'Hungria', 'Venezuela', 'Belgica', 'Chile', 'Líbano', 'Noruega', 'Puerto Rico', 'Reino Unido', 'Suiza', 'Japon', 'Polonia', 'Republica Dominicana', 'Oriente', 'Nicaragua', 'China', 'Nueva Zelanda', 'Escocia', 'Ecuador', 'Suecia', 'Egipto', 'El Salvador', 'Paraguay', 'Prusia', 'Costa Rica', 'Gales']
1


In [99]:
db1.columns

Index(['Título', 'Tiempo de Lectura (min)', 'Primer Nombre Autor',
       'Apellido Autor', 'Nombre Completo Autor', 'País', 'Género',
       'Año de nacimiento', 'Año de muerte', 'Autores relacionados',
       'Categorías', 'author_uuid', 'Imagen', 'Audio'],
      dtype='object')

## Creating Linked Json

In [100]:
# prompt: Create a JSON with db1['Author_name'] and db['linked_authors'] fitting an object {nodes, links} where linked authors are separated by comma and iterated on

import json

nodes = []
links = []
for i, row in db1.iterrows():
    author_name = row['Nombre Completo Autor']
    linked_authors = row['Autores relacionados'].split(',')

    # if nan: continue
    if author_name == 'Nan' or author_name == '':
        continue

    # if author name already in node
    if author_name in [node['id'] for node in nodes]:
        continue

    """
    'Título', 'Tiempo de Lectura (min)', 'Primer Nombre Autor',
       'Apellido Autor', 'Nombre Completo Autor', 'País', 'Género',
       'Año de nacimiento', 'Año de muerte', 'Autores relacionados',
       'Categorías', 'author_uuid', 'Imagen', 'Audio'
    """

    # Create a node for the author
    node = {
        'id': author_name,
        'uuid':str(row['author_uuid']),
        'group': get_country_index(row['País']),
        'country': str(row['País']),
        'birth_year': str(row['Año de nacimiento']),
        'death_year': str(row['Año de muerte']),
        'genre': str(row['Género']),
        'image': str(row['Imagen']),
        'audio': str(row['Audio']),
        'story_name':str(row['Título']),
        'linked_authors': len(linked_authors)
    }
    nodes.append(node)

    # Create links for each linked author
    for linked_author in linked_authors:
        link = {
            'source': author_name,
            'target': linked_author,
            'value': 1
        }
        links.append(link)

# Create the JSON object
data = {
    'nodes': nodes,
    'links': links
}

# Save the JSON object to a file
with open('authorLinksShortCleaned.json', 'w') as outfile:
    json.dump(data, outfile,ensure_ascii=False)


In [101]:
for i in range(10):
    print(data['nodes'][i])

for i in range(10):
    print(data['links'][i])

{'id': 'Pío Baroja', 'uuid': '9d50c449-ce8b-43f1-a0c1-53be4177dc93', 'group': 1, 'country': 'España', 'birth_year': '1872', 'death_year': '1956', 'genre': 'Romanticismo', 'image': 'Sagan.png', 'audio': 'tenquita.mp3', 'story_name': 'Las vocales de colores', 'linked_authors': 11}
{'id': 'Isaac Asimov', 'uuid': '8ccde480-27a6-40d7-9bb8-cd7fa5c965e2', 'group': 2, 'country': 'Estados Unidos', 'birth_year': '1920', 'death_year': '1992', 'genre': 'Romanticismo', 'image': 'Sagan.png', 'audio': 'tenquita.mp3', 'story_name': 'Cuánto se divertían', 'linked_authors': 2}
{'id': 'Mario Benedetti', 'uuid': '3599cfd0-b876-4ad4-b911-c4a75f73c236', 'group': 3, 'country': 'Uruguay', 'birth_year': '1920', 'death_year': '2009', 'genre': 'Romanticismo', 'image': 'benedetti.png', 'audio': 'tenquita.mp3', 'story_name': 'Todo lo contrario', 'linked_authors': 4}
{'id': 'Virgilio Pinera', 'uuid': '94fd67c5-724c-449b-b387-5de9c8ebe1fe', 'group': 4, 'country': 'Cuba', 'birth_year': '1912', 'death_year': '1979', '

In [102]:
# prompt: Go through the data and create a dataReduced only with the targets that exist in the nodes

dataReduced = {'nodes': [node for node in data['nodes']], 'links': []}

# Add links to dataReduced if their target exists in nodes
for link in data['links']:
    if link['target'] in [n['id'] for n in nodes]:
        dataReduced['links'].append(link)

# Print the reduced data
print(dataReduced)


{'nodes': [{'id': 'Pío Baroja', 'uuid': '9d50c449-ce8b-43f1-a0c1-53be4177dc93', 'group': 1, 'country': 'España', 'birth_year': '1872', 'death_year': '1956', 'genre': 'Romanticismo', 'image': 'Sagan.png', 'audio': 'tenquita.mp3', 'story_name': 'Las vocales de colores', 'linked_authors': 11}, {'id': 'Isaac Asimov', 'uuid': '8ccde480-27a6-40d7-9bb8-cd7fa5c965e2', 'group': 2, 'country': 'Estados Unidos', 'birth_year': '1920', 'death_year': '1992', 'genre': 'Romanticismo', 'image': 'Sagan.png', 'audio': 'tenquita.mp3', 'story_name': 'Cuánto se divertían', 'linked_authors': 2}, {'id': 'Mario Benedetti', 'uuid': '3599cfd0-b876-4ad4-b911-c4a75f73c236', 'group': 3, 'country': 'Uruguay', 'birth_year': '1920', 'death_year': '2009', 'genre': 'Romanticismo', 'image': 'benedetti.png', 'audio': 'tenquita.mp3', 'story_name': 'Todo lo contrario', 'linked_authors': 4}, {'id': 'Virgilio Pinera', 'uuid': '94fd67c5-724c-449b-b387-5de9c8ebe1fe', 'group': 4, 'country': 'Cuba', 'birth_year': '1912', 'death_ye

In [103]:
print(len(dataReduced['links']))
print(len(data['links']))


635
1506


In [104]:
# Save the JSON object to a file
with open('authorLinksSmallerShortCleaned.json', 'w') as outfile:
    json.dump(dataReduced, outfile,ensure_ascii=False)

## Story Embeddings

In [ ]:
db3.head(1)

,dim1,dim2,dim3,vector_id,uuid_story,author_uuid,story_name,link,length,words,reading_time_min,coordinates
0,"16,28848076","-8,153986931","8,296456337",04d4cd4b-8f57-4910-8cff-cfd34ca6bf46,04d4cd4b-8f57-4910-8cff-cfd34ca6bf46,9d50c449-ce8b-43f1-a0c1-53be4177dc93,Las vocales de colores,https://ciudadseva.com/texto/las-vocales-de-co...,1765,305,"1,525","[16.28848076, -8.153986931, 8.296456337]"


In [ ]:
# prompt: convert db3 cols dim1, dim2, dim3 into a new one with float coordinates. They are currently strings like "16,0566546", so replace , with . and eval
df = pd.DataFrame()

df['story_name'] = db3['story_name']
df['x'] = [ float(d.replace(',', '.')) for d in db3['dim1'] ]
df['y'] = [ float(d.replace(',', '.')) for d in db3['dim2'] ]
df['z'] = [ float(d.replace(',', '.')) for d in db3['dim3'] ]
df.head()

,story_name,x,y,z,size
0,Las vocales de colores,16.288481,-8.153987,8.296456,0.1
1,Cuánto se divertían,16.784241,-6.143554,10.079655,0.1
2,Todo lo contrario,19.959745,-9.638548,8.536081,0.1
3,La carne,18.058767,-8.713723,10.635745,0.1
4,La rana gritona y el león,19.603294,-7.778275,12.218028,0.1


In [ ]:
import plotly.express as px

# Big figure plotly
fig = px.scatter_3d(df, x='x', y='y', z='z',text='story_name')
fig.update_traces(mode = 'markers')
fig.show()

In [ ]:
fig = px.scatter(df,'x','y',text='story_name')
fig.update_traces(mode = 'markers')